In [4]:
include("astrobee_se3_script.jl")
using Plots

N = 1000
MAX_ITERATIONS_NB = 1e4

model = Astrobee()

Xp,Up = initialize_trajectory(model, N)
X, U  = copy(Xp) .+ 2., copy(Up) .+ 2.

SCPproblem = GuSTOProblem(model, N, Xp, Up)

x_dim, u_dim = model.x_dim, model.u_dim

Delta0, omega0, omegamax, 
    epsilon, rho0, rho1, 
    beta_succ, beta_fail, gamma_fail, 
    convergence_threshold = get_initial_gusto_parameters(model)

Delta = Delta0
omega = omega0

# X = zeros(x_dim, N)
# U = zeros(u_dim, N-1)
println("oh yeah")

oh yeah


In [5]:
include("astrobee_se3_script.jl")

X_all, U_all, x_shooting_all = [], [], []
push!(X_all, copy(X))
push!(U_all, copy(U))

B_success = false
it = 0

while it<MAX_ITERATIONS_NB && 
        !(it!=0 && it!=1 && it!=2 && it!=3 && B_success && 
            convergence_metric(model,X,U,Xp,Up)<convergence_threshold) &&
        omega<omegamax
    println("-----------")
    println("Iteration $it")
    println("metric=$(convergence_metric(model,X,U,Xp,Up))")
    println("-----------")
    
    
#     omega, Delta = SCPproblem.omega, SCPproblem.Delta
    
    Xp = copy(X)
    Up = copy(U)
    
    model.f, model.A, model.B = compute_dynamics(model, Xp, Up)
    
    reset_problem(SCPproblem, model)
    set_parameters(SCPproblem, model, Xp, Up, omega, Delta)
    define_cost(SCPproblem, model)
    define_constraints(SCPproblem, model)
    
    JuMP.optimize!(SCPproblem.solver_model)    
    p0 = get_initial_constraint_dual_variable(SCPproblem, model)
    println("dual variables:", p0)
    # get solution
    X_sol = JuMP.value.(SCPproblem.X)
    U_sol = JuMP.value.(SCPproblem.U)
    
    # try shooting
    #x_shooting, sol_newton = solve_shooting(SCPproblem, model, p0)
#     x_shooting = solve_shooting_once(SCPproblem, model, p0)
    #push!(x_shooting_all, x_shooting)
    
    # -----------
    # GuSTO Logic
    if it > 3
        if is_in_trust_region(model, X_sol, U_sol, Xp, Up, Delta)
            rho = accuracy_ratio(SCPproblem, model, X_sol, U_sol, Xp, Up)

            if rho > rho1
                println("Reject solution.")
                Delta = beta_fail * Delta
                omega     = omega
                B_success = false

            else
                println("Accept solution.")
                X = copy(X_sol)
                U = copy(U_sol)
                B_success = true
                if rho < rho0
                    Delta = min(beta_succ*Delta, Delta0)
                else
                    Delta = Delta
                end
                if satisfies_state_inequality_constraints(SCPproblem, model, X_sol, U_sol, Xp, Up, Delta)
                    omega = omega0
                else
                    println("Solution does not satisfy state constraints, increasing omega.")
                    omega = gamma_fail * omega
                end
            end
        else
            println("Reject solution (Outside trust region)")
            Delta = Delta
            omega     = gamma_fail * omega
            B_success = false
        end
#         if convergence_metric(model,X,U,Xp,Up) <0.6
#             println("Convergence metric very small. Decreasing Delta.")
#             Delta = Delta0./(2^it)
#         end
            
    else # always accept first solution
        X = copy(X_sol)
        U = copy(U_sol)
    end
        
                    
    # -----------

    push!(X_all, copy(X))
    push!(U_all, copy(U))
                    
    
    it += 1
    
    diff_with_prev = norm(copy(X) - copy(Xp), Inf)
    println("x(k) - x(k-1) = $diff_with_prev")
    println("Parameters:")
    println("omega=$omega")
    println("delta=$Delta")
end
println("Converged")

-----------
Iteration 0
metric=94.24868789068172
-----------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

dual variables:[-1675.84, -2463.16, -2111.94, -1105.2, -1510.03, -1329.76, -45.4357, -13.4329, -19.7999, -16.5666, -2.24876, -4.81588, -2.72668]
x(k) - x(k-1) = 2.712491572623643
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 1
metric=142.99580730744955
-----------
dual variables:[-71.0568, -96.3085, -84.8343, -211.353, -252.865, -234.263, -116.223, 23.6071, 21.0808, 18.8167, -1.66821, 0.0277189, 6.74868]
x(k) - x(k-1) = 2.1782771826693628
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 2
metric=64.34707097023932
-----------
d

dual variables:[-3.45502, -3.29505, -3.37587, -30.8405, -28.3372, -29.5739, -0.49339, -0.102474, 0.163812, -0.00300377, -0.14647, -0.125739, 0.569557]
Accept solution.
x(k) - x(k-1) = 0.027018560683750747
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 28
metric=0.7386041088332757
-----------
dual variables:[-3.35986, -3.2, -3.2806, -30.2774, -27.7792, -29.0125, -0.475688, -0.1058, 0.16068, -0.00275233, -0.14093, -0.126748, 0.552476]
Accept solution.
x(k) - x(k-1) = 0.025356324894157067
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 29
metric=0.7018808782606211
-----------
dual variables:[-3.27062, -3.111, -3.19134, -29.7444, -27.2518, -28.4817, -0.45928, -0.10852, 0.157749, -0.00252792, -0.135831, -0.127473, 0.536593]
Accept solution.
x(k) - x(k-1) = 0.023832146940787835
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 30
metric=0.6677933840711102
-----------
dual variables:[-3.18674, -3.02748, -3.10751, -29.2389, -26.7524, -27.9786, -0.444048, -0.110713, 0.

dual variables:[-2.03747, -1.89628, -1.96604, -21.7714, -19.4579, -20.5914, -0.262286, -0.102105, 0.107062, -0.000494906, -0.0712933, -0.109729, 0.337335]
Accept solution.
x(k) - x(k-1) = 0.008426253225762514
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 55
metric=0.3423898511313849
-----------
dual variables:[-2.00927, -1.86887, -1.93821, -21.5727, -19.2661, -20.3961, -0.2586, -0.100959, 0.10546, -0.000465646, -0.0698556, -0.108609, 0.33334]
Accept solution.
x(k) - x(k-1) = 0.00816664259468819
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 56
metric=0.33588418726954544
-----------
dual variables:[-1.98195, -1.84231, -1.91126, -21.3795, -19.0796, -20.2061, -0.255064, -0.099802, 0.10388, -0.00044628, -0.0684603, -0.10749, 0.329495]
Accept solution.
x(k) - x(k-1) = 0.007917974445388443
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 57
metric=0.3296080564396532
-----------
dual variables:[-1.95548, -1.81661, -1.88515, -21.1914, -18.8983, -20.0213, -0.251675,

dual variables:[-1.49742, -1.37463, -1.43494, -17.7967, -15.6454, -16.6967, -0.197253, -0.0731869, 0.0717336, 2.78144e-5, -0.0431699, -0.0823189, 0.264422]
Accept solution.
x(k) - x(k-1) = 0.005347737649709083
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 82
metric=0.22385060015320246
-----------
dual variables:[-1.48361, -1.3614, -1.42142, -17.6895, -15.5433, -16.5921, -0.195693, -0.072278, 0.070709, 3.83786e-5, -0.04243, -0.0814591, 0.26261]
Accept solution.
x(k) - x(k-1) = 0.005280876199552642
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 83
metric=0.22105347345749568
-----------
dual variables:[-1.4701, -1.34847, -1.40819, -17.5844, -15.4433, -16.4895, -0.194169, -0.0713821, 0.0697018, 4.85296e-5, -0.0417052, -0.080611, 0.260836]
Accept solution.
x(k) - x(k-1) = 0.005215980261667141
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 84
metric=0.2183292281827607
-----------
dual variables:[-1.45689, -1.33582, -1.39526, -17.4812, -15.3452, -16.3889, -0.192

dual variables:[-1.20878, -1.09972, -1.15314, -15.4723, -13.4441, -14.4341, -0.164404, -0.052992, 0.049405, 0.000111971, -0.0275057, -0.0629121, 0.225723]
Accept solution.
x(k) - x(k-1) = 0.004034133202523549
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 109
metric=0.16917952118075671
-----------
dual variables:[-1.20067, -1.09204, -1.14525, -15.404, -13.3797, -14.3678, -0.163452, -0.0523915, 0.0487543, 0.000113921, -0.0270608, -0.0623266, 0.224591]
Accept solution.
x(k) - x(k-1) = 0.004000088926445766
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 110
metric=0.16776965732534554
-----------
dual variables:[-1.1927, -1.08449, -1.13749, -15.3367, -13.3163, -14.3024, -0.162515, -0.0518001, 0.0481141, 0.00011576, -0.0266234, -0.0617493, 0.223474]
Accept solution.
x(k) - x(k-1) = 0.003966748873964532
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 111
metric=0.16639170428536615
-----------
dual variables:[-1.18486, -1.07708, -1.12987, -15.2703, -13.2537, -14.23

...

dual variables:[-0.196575, -0.173792, -0.18495, -4.87418, -3.90178, -4.37585, -0.019309, -0.00140601, -0.00226222, 2.08557e-5, 0.00687581, -0.00500397, 0.0406938]
Accept solution.
x(k) - x(k-1) = 0.0003484413189291935
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 1596
metric=0.01621307095806889
-----------
dual variables:[-0.196534, -0.173764, -0.184915, -4.8733, -3.90108, -4.37506, -0.0192985, -0.0014054, -0.0022619, 2.08454e-5, 0.00687492, -0.00500263, 0.040677]
Accept solution.
x(k) - x(k-1) = 0.00034838207983023395
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 1597
metric=0.016212984747577333
-----------
dual variables:[-0.196493, -0.173735, -0.184881, -4.87242, -3.90038, -4.37427, -0.019288, -0.00140479, -0.00226158, 2.08351e-5, 0.00687403, -0.00500129, 0.0406603]
Accept solution.
x(k) - x(k-1) = 0.0003483228206184741
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 1598
metric=0.01621289689721482
-----------
dual variables:[-0.196452, -0.173707, -0.1

dual variables:[-0.195511, -0.173054, -0.184053, -4.85133, -3.8836, -4.35541, -0.0190393, -0.00139048, -0.00225382, 2.0591e-5, 0.00685272, -0.00496968, 0.0402636]
Accept solution.
x(k) - x(k-1) = 0.0003468955745260516
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 1622
metric=0.016210316950202242
-----------
dual variables:[-0.195471, -0.173026, -0.184019, -4.85046, -3.8829, -4.35463, -0.0190291, -0.0013899, -0.0022535, 2.0581e-5, 0.00685183, -0.00496838, 0.0402473]
Accept solution.
x(k) - x(k-1) = 0.00034683601412277554
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 1623
metric=0.016210190629085904
-----------
dual variables:[-0.19543, -0.172997, -0.183985, -4.84959, -3.8822, -4.35384, -0.0190189, -0.00138932, -0.00225317, 2.0571e-5, 0.00685095, -0.00496709, 0.040231]
Accept solution.
x(k) - x(k-1) = 0.00034677643778957545
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 1624
metric=0.016210062865389282
-----------
dual variables:[-0.195389, -0.172969, -0.1

Excessive output truncated after 524289 bytes.

[-0.194463, -0.17232, -0.183167, -4.82872, -3.86549, -4.33512, -0.0187777, -0.00137559, -0.00224528, 2.03337e-5, 0.00682974, -0.0

InterruptException: InterruptException:

In [13]:
println("i is  $it")
println("metric=$(convergence_metric(model,X,U,Xp,Up))")
diff_with_prev = norm(copy(X) - copy(Xp), Inf)
println("x(k) - x(k-1) = $diff_with_prev")
    println("Parameters:")
    println("omega=$omega")
    println("delta=$Delta")
p0 = get_initial_constraint_dual_variable(SCPproblem, model)
println("dual variables:", p0)

i is  3304
metric=0.0
x(k) - x(k-1) = 0.0
Parameters:
omega=1.0
delta=1000.0
dual variables:[-0.143969, -0.129436, -0.136593, -3.67809, -2.82725, -3.24293, -0.0102765, -0.000960137, -0.00174258, 1.1633e-5, 0.0057055, -0.00390109, 0.0253608]


In [6]:
gamma_fail

5.0

In [7]:
init_duals = get_initial_constraint_dual_variable(SCPproblem, model)

13-element Array{Float64,1}:
 -0.14396906226616943  
 -0.1294360041212453   
 -0.13659329062223785  
 -3.6780949537574155   
 -2.8272547692658656   
 -3.242925123954885    
 -0.010276468813509538 
 -0.0009601366937471721
 -0.0017425805362279252
  1.1633031378484973e-5
  0.0057055039402512785
 -0.0039010946455757005
  0.025360791013570933 

In [ ]:
include("astrobee_se3_script.jl")
 
sol_newton = solve_shooting(SCPproblem, model, get_initial_constraint_dual_variable(SCPproblem, model))
#x_shooting = solve_shooting_once(SCPproblem, model, sol_newton.zero)

In [ ]:
xN = x_shooting[:,end]

for i = 1:x_dim
    Fi = model.x_final[i] - xN[i]
    @show Fi
end

In [ ]:

    for x_shooting in x_shooting_all
        if size(x_shooting,1) > 1
        @show minimum(x_shooting)
        end
end

In [ ]:
x_shooting_all

In [ ]:
include("astrobee_se3_script.jl")
#plot(X[1,:],X[2,:])

plot_solutions(SCPproblem, model, X_all[end-1:end], U_all[end-1:end]; x_shooting_all=x_shooting_all)

In [ ]:
X = X_all[end]
# Xs = x_shooting_all[2]
 fig = plot(X[1,:], X[2,:], X[3,:]; c=:blue, lims = (-0.7,0.7))
# plot!(fig, Xs[1,:], Xs[2,:], Xs[3,:]; c=:red)

xlims = (0,10)
zlims = (0,10)
fig

In [ ]:
idx = [1,2]
x = X_all[2][1,:]
y = X_all[2][2,:]
z = X_all[2][3,:]
# plot(x,  y)
plot(1:N, y)